In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
package_dir = "../input/pretrained-models.pytorch-master/"
sys.path.insert(0, package_dir)
import pretrainedmodels
package_dir = '../input/early-stopping-pytorch'
sys.path.append(package_dir)
from pytorchtools import EarlyStopping
import numpy as np
import pandas as pd
import scipy as sp
from functools import partial
from sklearn import metrics
from sklearn.model_selection import KFold
from collections import Counter
import json
import time
import cv2
import albumentations
from albumentations import torch as AT
import gc
import torchvision
import torch.nn as nn
from tqdm import tqdm
#from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device=device)
#ImageFile.LOAD_TRUNCATED_IMAGES = True

# To have reproducible results and compare them
nr_seed = 2019
import numpy as np 
np.random.seed(nr_seed)

# Specify GPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
device = torch.device("cuda:0")

# pre-processing

In [ ]:
def crop_image1(img,tol=7):
    # tol  is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img 
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        
        return img

In [ ]:
transform = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.HorizontalFlip(),
    albumentations.RandomBrightness(),
    albumentations.JpegCompression(80),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])
transform_valid_test = albumentations.Compose([
    albumentations.Resize(224, 224),
    albumentations.Normalize(),
    AT.ToTensor()
    ])


In [ ]:
# dfのimage_idから対応する画像を読み込んだり、前処理をかけたりする関数
class RetinopathyDataset(Dataset):
    def __init__(self, csv_file, transform, datatype='train'):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.datatype = datatype

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.datatype=='train':
            img_name = os.path.join('../input/aptos2019-blindness-detection/train_images',
                                self.data.loc[idx, 'id_code'] + '.png')
        elif self.datatype=='train_old':
            img_name = os.path.join('../input/diabetic-retinopathy-resized/resized_train',
                                self.data.loc[idx, 'image'] + '.jpeg')
        else:
            img_name =  os.path.join('../input/aptos2019-blindness-detection/test_images',
                                     self.data.loc[idx, 'id_code'] + '.png')   
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.circle_crop(img, sigmaX=10)
        img = self.transform(image=img)
        img = img['image']
        if self.datatype=='train':
            label = torch.tensor(self.data.loc[idx, 'diagnosis'])
            return {'image': img,
                    'labels': label
                    }
        elif self.datatype=='train_old':
            label = torch.tensor(self.data.loc[idx, 'level'])
            return {'image': img,
                    'labels': label
                    }
        else:
            return {'image': img}
    def circle_crop(self, img, sigmaX=10):   
        height, width, depth = img.shape    

        x = int(width/2)
        y = int(height/2)
        r = np.amin((x,y))

        circle_img = np.zeros((height, width), np.uint8)
        cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
        img = cv2.bitwise_and(img, img, mask=circle_img)
        img = crop_image_from_gray(img)
        img = cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
        return img 
    

## kappa_lossにとる最適化をするclass。使い方勉強中
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
def load_model(pretrained='imagenet'):
    model = pretrainedmodels.__dict__['resnet101'](pretrained=pretrained)
    model.avg_pool = nn.AdaptiveAvgPool2d(1)
    model.last_linear = nn.Sequential(
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.25),
                              nn.Linear(in_features=2048, out_features=2048, bias=True),
                              nn.ReLU(),
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.5),
                              nn.Linear(in_features=2048, out_features=1, bias=True),
                             )
    return model

# train

In [ ]:
class train(object):
    def __init__(self, params):
        self.lr = params.get('lr')
        self.img_size = params.get('img_size')
        self.batch_size = params.get('batch_size')
        self.n_epochs = params.get('n_epochs')
        self.n_freeze = params.get('n_freeze')
        self.coef = params.get('coef')
        self.criterion = params.get('criterion')
        self.num_workers = params.get('num_workers')
        self.load_state = params.get('load_state')
        self.load_path = params.get('load_path')
        self.save_path = params.get('save_path')
        self.device = params.get('device')
        self.n_folds = params.get('n_folds')
        self.use_valid = False
        self.early_stop = params.get("early_stop"),
        self.patience = params.get("patience"),
    
    def get_train(self,data):
        self.train_data = data
        
    def get_valid(self,data):
        self.valid_data = data
        self.use_valid = True
        
    def fit(self, use_cv=False, train_idx=None, valid_idx=None):
        since = time.time()
        # Model
        if self.load_state:
            model = load_model(pretrained=None)
            model.load_state_dict(torch.load(self.load_path, map_location=self.device))
        else:
            model = load_model(pretrained='imagenet')
        model = model.to(device)

        
        plist = [{'params': model.parameters(), 'lr': self.lr}]
        optimizer = optim.Adam(plist, lr=self.lr)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
        
        if use_cv:
            train_sampler = SubsetRandomSampler(train_idx)
            valid_sampler = SubsetRandomSampler(valid_idx)
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers,sampler=train_sampler)
            data_loader_valid = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers,sampler=valid_sampler)
        elif self.use_valid:
            print("OK")
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
            data_loader_valid = torch.utils.data.DataLoader(self.valid_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
        else:
            data_loader_train = torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size,
                                                num_workers=self.num_workers)
            
        if self.early_stop:
            early_stopping = EarlyStopping(patience=self.patience, verbose=True)
        
        for epoch in range(self.n_epochs):
            if epoch == self.n_freeze:      
                for param in model.parameters():
                    param.requires_grad = True

            print('Epoch {}/{}'.format(epoch, self.n_epochs - 1))
            print('-' * 10)
            scheduler.step()
            model.train()
            running_loss = 0.0
            tk0 = tqdm(data_loader_train, total=int(len(data_loader_train)))
            counter = 0
            for bi, d in enumerate(tk0):
                inputs = d["image"]
                labels = d["labels"].view(-1, 1)
                inputs = inputs.to(self.device, dtype=torch.float)
                labels = labels.to(self.device, dtype=torch.float)
                optimizer.zero_grad()
                with torch.set_grad_enabled(True):
                    outputs = model(inputs)
                    loss = self.criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                counter += 1
                tk0.set_postfix(loss=(running_loss / (counter * data_loader_train.batch_size)))
            epoch_loss = running_loss / len(data_loader_train)
            print('Training Loss: {:.4f}'.format(epoch_loss))
            
            if self.use_valid:
                model.eval()
                eval_loss = 0
                nb_eval_steps = 0

                for step, batch in enumerate(tqdm(data_loader_valid , total=int(len(data_loader_valid)))):

                    inputs = batch["image"]
                    labels = batch["labels"].view(-1, 1)

                    inputs = inputs.to(self.device, dtype=torch.float)
                    labels = labels.to(self.device, dtype=torch.float)

                    with torch.no_grad():
                        outputs = model(inputs)

                    y_hat = torch.Tensor.cpu(outputs.view(-1))
                    y = torch.Tensor.cpu(labels.view(-1))

                    for pred in enumerate(y_hat):
                        if pred[1] < self.coef[0]:
                            y_hat[1] = 0
                        elif pred[1] >= self.coef[0] and pred[1] < self.coef[1]:
                            y_hat[1] = 1
                        elif pred[1] >= self.coef[1] and pred[1] < self.coef[2]:
                            y_hat[1] = 2
                        elif pred[1] >= self.coef[2] and pred[1] < self.coef[3]:
                            y_hat[1] = 3
                        else:
                            y_hat[1] = 4

                    tmp_eval_loss = quadratic_kappa(y_hat, y)

                    eval_loss += tmp_eval_loss.mean().item()
                    nb_eval_steps += 1

                eval_loss = eval_loss / nb_eval_steps

                print('Validation Kappa: {:.4f}'.format(eval_loss))

                if self.early_stop:
                    eval_loss = 1 - eval_loss
                    early_stopping(eval_loss, model)
                    if early_stopping.early_stop:
                        print("Early stopping")
                        break

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        if not use_cv:
            torch.save(model.state_dict(), self.save_path)

        del(model, data_loader_train)
        gc.collect()
        torch.cuda.empty_cache() 
        
    def cv(self):
        num_train = len(self.train_data)
        indices = list(range(num_train))
        kf = KFold(n_splits=self.n_folds, random_state=1337, shuffle=True)

        train_idx = []
        valid_idx = []

        for t, v in kf.split(indices):
            train_idx.append(t)
            valid_idx.append(v)

        # Training                        
        for fold in np.arange(self.n_folds):
            print('Fold:',fold)
            self.fit(use_cv=True,train_idx=train_idx[fold],valid_idx=valid_idx[fold])

In [ ]:
#params
params = {
    "lr" : 1e-5,
    "img_size" : 224,
    "batch_size" : 48,
    "n_epochs" : 1,
    "n_freeze" : 1,
    "num_workers" : 8,
    "coef" : [0.5, 1.5, 2.5, 3.5],
    "criterion" : nn.MSELoss(),
    "load_state" : False,# fine-tuning済みのモデルを使うか
    "load_path" : None, 
    'save_path' : None,
    "device" : device,
    "n_folds" : 2,
    "early_stop" : True,
    "patience" : 2,
}

#training for the lazy, like me

# cv: new
if 0:
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                            transform=transform, datatype='train'))
    Mytrain.cv()

# train:old & valid:new
elif 0:
    
    params['save_path'] =  "../input/mmmodel/model_old_test.bin"
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/diabetic-retinopathy-resized/trainLabels.csv",
                                            transform=transform, datatype='train_old'))
    Mytrain.get_valid(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                           transform=transform, datatype='train'))
    Mytrain.fit()
    
# train:new
elif 1:
    params['n_epochs'] = 1
    params['load_state'] = True
    params['load_path'] =  "../input/mmmodel/model_old_test.bin" 
    params['save_path'] =  "../input/mmmodel/model_new_test.bin"        
    Mytrain = train(params)
    Mytrain.get_train(RetinopathyDataset(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                           transform=transform, datatype='train'))
    Mytrain.fit()

# Inference

In [ ]:
model = load_model(pretrained=None)
model.load_state_dict(torch.load("../input/mmmodel/model_0726_01.bin", map_location=device))
model.eval()
model = model.to(device)

In [ ]:
test_dataset = RetinopathyDataset(csv_file='../input/aptos2019-blindness-detection/sample_submission.csv',
                                      transform=transform_valid_test, datatype='test')

In [ ]:
test_bs = 32
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False, num_workers=4)
test_preds = np.zeros((len(test_dataset), 1))
tk0 = tqdm(test_data_loader)
for i, x_batch in tqdm(enumerate(tk0)):
    x_batch = x_batch["image"]
    pred = model(x_batch.to(device))
    test_preds[i * test_bs:(i + 1) * test_bs] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)

In [ ]:
sample["diagnosis"].value_counts()